In [67]:
from google.colab import drive
drive.mount('./Mbako')

Drive already mounted at ./Mbako; to attempt to forcibly remount, call drive.mount("./Mbako", force_remount=True).


In [68]:
#IMPORT PACKAGE
from PIL import Image
import pathlib
import numpy as np
import os
import zipfile
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
#from tensorflow.keras.utils import img_to_array, load_img

In [69]:
#MAKE TRAIN AND VALIDATION DIRECTORY

#DON'T FORGET TO CHANGE THE DIRECTORY PATH
train_dir = "./Mbako/MyDrive/Project Bangkit/Dataset Tembakau resize/Training/"
train_rendah_dir = os.path.join(train_dir, "Rendah/")
train_sedang_dir = os.path.join(train_dir, "Sedang/")
train_tinggi_dir = os.path.join(train_dir, "Tinggi/")
train_unknown_dir = os.path.join(train_dir, "Unknown/")

#DON'T FORGET TO CHANGE THE DIRECTORY PATH
validation_dir = "./Mbako/MyDrive/Project Bangkit/Dataset Tembakau resize/Validation/"
validation_rendah_dir = os.path.join(validation_dir, "Rendah/")
validation_sedang_dir = os.path.join(validation_dir, "Sedang/")
validation_tinggi_dir = os.path.join(validation_dir, "Tinggi/")
validation_unknown_dir = os.path.join(validation_dir, "Unknown/")

In [70]:
# Load the first example of a Garangan Tobacco low class
sample_image  = load_img(f"{os.path.join(train_rendah_dir, os.listdir(train_rendah_dir)[0])}")

# Convert the image into its numpy array representation
sample_array = img_to_array(sample_image)

print(f"Each image has shape: {sample_array.shape}")

print(f"The maximum pixel value used is: {np.max(sample_array)}")



Each image has shape: (300, 300, 3)
The maximum pixel value used is: 255.0


In [71]:
def image_generator(train_dir,validation_dir):
    # Instantiate the ImageDataGenerator class.
    # Remember to set the rescale argument.
    train_datagen = ImageDataGenerator(rescale=1/255,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode="nearest")

    # Specify the method to load images from a directory and pass in the appropriate arguments:
    # - directory: should be a relative path to the directory containing the data
    # - targe_size: set this equal to the resolution of each image (excluding the color dimension)
    # - batch_size: number of images the generator yields when asked for a next batch. Set this to 10.
    # - class_mode: How the labels are represented. Should be one of "binary", "categorical" or "sparse".
    #               Pick the one that better suits here given that the labels are going to be 1D binary labels.
    train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                        target_size=(224, 224),
                                                        batch_size=16,
                                                        class_mode="categorical")
    
    # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
    # Remember that validation data should not be augmented
    validation_datagen = ImageDataGenerator( rescale = 1/255 )

    # Pass in the appropriate arguments to the flow_from_directory method
    validation_generator = validation_datagen.flow_from_directory(directory=validation_dir,
                                                                 batch_size=16, 
                                                                 class_mode='categorical',
                                                                 target_size=(224, 224))
    ### END CODE HERE

    return train_generator,validation_generator

In [72]:
train_generator, validation_generator = image_generator(train_dir, validation_dir)

Found 215 images belonging to 4 classes.
Found 61 images belonging to 4 classes.


In [73]:
# # Import the inception model  
# from tensorflow.keras.applications.inception_v3 import InceptionV3

# # Create an instance of the inception model from the local pre-trained weights
# #DON'T FORGET TO CHANGE THE DIRECTORY PATH
# local_weights_file = './Mbako/MyDrive/Mbako/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
from tensorflow.keras.applications.mobilenet import MobileNet

# Load the VGG16 model
pre_trained_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')


In [74]:
# create_pre_trained_model

# def create_pre_trained_model(local_weights_file):
#   """
#   Initializes an InceptionV3 model.
  
#   Args:
#     local_weights_file (string): path pointing to a pretrained weights H5 file
    
#   Returns:
#     pre_trained_model: the initialized InceptionV3 model
#   """
#   ### START CODE HERE
#   pre_trained_model = MobileNet(input_shape = (300, 300, 3),
#                                   include_top = False, 
#                                   weights = None) 

#   pre_trained_model.load_weights(local_weights_file)

#   # Make all the layers in the pre-trained model non-trainable
#   for layer in pre_trained_model.layers:
#     layer.trainable = False

#   ### END CODE HERE

#   return pre_trained_model

In [75]:
# pre_trained_model = create_pre_trained_model(local_weights_file)

# Print the model summary
pre_trained_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [76]:
# Define a Callback class that stops training once accuracy reaches 95%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy')>0.95 and logs.get('accuracy')>0.95):
      print("\nReached 95% accuracy so cancelling training!")
      self.model.stop_training = True

In [77]:
# output_of_last_layer

def output_of_last_layer(pre_trained_model):
  """
  Gets the last layer output of a model
  
  Args:
    pre_trained_model (tf.keras Model): model to get the last layer output from
    
  Returns:
    last_output: output of the model's last layer 
  """
  ### START CODE HERE
  last_desired_layer = pre_trained_model.get_layer('conv_dw_13_relu')#conv_dw_10_relu
  print('last layer output shape: ', last_desired_layer.output_shape)
  last_output = last_desired_layer.output
  print('last layer output: ', last_output)
  ### END CODE HERE

  return last_output

In [78]:
last_output = output_of_last_layer(pre_trained_model)

last layer output shape:  (None, 7, 7, 1024)
last layer output:  KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 1024), dtype=tf.float32, name=None), name='conv_dw_13_relu/Relu6:0', description="created by layer 'conv_dw_13_relu'")


In [79]:
# Print the type of the pre-trained model
print(f"The pretrained model has type: {type(pre_trained_model)}")

The pretrained model has type: <class 'keras.engine.functional.Functional'>


In [80]:
def create_uncompiled_model():

    ### START CODE HERE
    
    # Flatten the output layer to 1 dimension
    x = layers.Flatten()(last_output)

    ### START CODE HERE

    # Add a fully connected layer with 1024 hidden units and ReLU activation
    x = layers.Dense(256, activation='relu')(x)
    # Add a dropout rate of 0.2
    x = layers.Dropout(0.2)(x)       
    # Add a final sigmoid layer for classification
    x = x = layers.Dense  (3, activation='softmax')(x)         

    # Create the complete model by using the Model class
    model = Model(inputs=pre_trained_model.input, outputs=x)
    
    ### END CODE HERE

    return model

In [81]:
# Test your uncompiled model
uncompiled_model = create_uncompiled_model()

try:
    uncompiled_model.predict(train_generator)
except:
    print("Your current architecture is incompatible with the windowed dataset, try adjusting it.")
else:
    print("Your current architecture is compatible with the windowed dataset! :)")

14/14 [==============================] - 4s 238ms/step
Your current architecture is compatible with the windowed dataset! :)


In [82]:
def adjust_learning_rate(dataset):
    
    model = create_uncompiled_model()
    
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-5 * 10**(epoch / 20))
    
    ### START CODE HERE
    
    # Select your optimizer
    optimizer = tf.keras.optimizers.Adam()
    
    # Compile the model passing in the appropriate loss
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer, 
                  metrics=["accuracy"]) 
    
    ### END CODE HERE
    
    history = model.fit(train_generator, epochs=200, callbacks=[lr_schedule])
    
    return history

In [83]:
'''# Run the training with dynamic LR
lr_history = adjust_learning_rate(train_generator)'''

'# Run the training with dynamic LR\nlr_history = adjust_learning_rate(train_generator)'

In [84]:
'''plt.semilogx(lr_history.history["lr"], lr_history.history["loss"])
plt.axis([1e-4, 10, 0, 10])'''

'plt.semilogx(lr_history.history["lr"], lr_history.history["loss"])\nplt.axis([1e-4, 10, 0, 10])'

In [85]:
#create_final_model

def create_final_model(pre_trained_model, last_output):
  """
  Appends a custom model to a pre-trained model
  
  Args:
    pre_trained_model (tf.keras Model): model that will accept the train/test inputs
    last_output (tensor): last layer output of the pre-trained model
    
  Returns:
    model: the combined model
  """
  # SAMAKAN MODEL FINAL DENGAN MODEL UNCOMPILED
  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(last_output)

  ### START CODE HERE

  # Add a fully connected layer with 1024 hidden units and ReLU activation
  x = layers.Dense(256, activation='relu')(x)
  # Add a dropout rate
  x = layers.Dropout(0.2)(x)       
  # Add a final sigmoid layer for classification
  x = x = layers.Dense(4, activation='softmax')(x)         

  # Create the complete model by using the Model class
  model = Model(inputs=pre_trained_model.input, outputs=x)

  # Compile the model
  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), 
                loss = 'categorical_crossentropy',
                metrics = ['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

  ### END CODE HERE
  
  return model

In [86]:
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [87]:
# Save your model in a variable
model = create_final_model(pre_trained_model, last_output)

# Inspect parameters
total_params = model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in model.trainable_weights])

print(f"There are {total_params:,} total parameters in this model.")
print(f"There are {num_trainable_params:,} trainable parameters in this model.")

There are 15,022,532 total parameters in this model.
There are 15,002,692 trainable parameters in this model.


In [88]:
callbacks = myCallback()
history = model.fit(train_generator,
                    validation_data = validation_generator,
                    epochs = 300,
                    callbacks=callbacks,
                    verbose = 1)

Epoch 1/300
14/14 [==============================] - 21s 439ms/step - loss: 52.4172 - accuracy: 0.3814 - precision_4: 0.3857 - recall_4: 0.3767 - val_loss: 36.9612 - val_accuracy: 0.6066 - val_precision_4: 0.6066 - val_recall_4: 0.6066
Epoch 2/300
14/14 [==============================] - 4s 283ms/step - loss: 18.5747 - accuracy: 0.5442 - precision_4: 0.5442 - recall_4: 0.5442 - val_loss: 8.2462 - val_accuracy: 0.6557 - val_precision_4: 0.6557 - val_recall_4: 0.6557
Epoch 3/300
14/14 [==============================] - 4s 294ms/step - loss: 5.8307 - accuracy: 0.6233 - precision_4: 0.6262 - recall_4: 0.6233 - val_loss: 15.5990 - val_accuracy: 0.5574 - val_precision_4: 0.5574 - val_recall_4: 0.5574
Epoch 4/300
14/14 [==============================] - 6s 404ms/step - loss: 3.5380 - accuracy: 0.6977 - precision_4: 0.6977 - recall_4: 0.6977 - val_loss: 20.2400 - val_accuracy: 0.5574 - val_precision_4: 0.5574 - val_recall_4: 0.5574
Epoch 5/300
14/14 [==============================] - 4s 287ms/

In [89]:
predictions = model.predict(train_generator)

predicted_labels = np.argmax(predictions, axis=1)

true_labels=train_generator.classes

confusion_matrix = tf.math.confusion_matrix(true_labels, predicted_labels)

# Extract the true positives, true negatives, and false positives
true_positives = np.diag(confusion_matrix)
true_negatives = np.sum(confusion_matrix) - np.sum(confusion_matrix, axis=0) - np.sum(confusion_matrix, axis=1) + true_positives
false_positives = np.sum(confusion_matrix, axis=0) - true_positives
false_negatives = np.sum(confusion_matrix, axis=1) - true_positives

# Print the results for each class
for i in range(3):
    print("Class", i)
    print("True Positives:", true_positives[i])
    print("True Negatives:", true_negatives[i])
    print("False Positives:", false_positives[i])
    print("False Negatives:", false_negatives[i])

14/14 [==============================] - 5s 300ms/step
Class 0
True Positives: 12
True Negatives: 133
False Positives: 34
False Negatives: 36
Class 1
True Positives: 7
True Negatives: 129
False Positives: 40
False Negatives: 39
Class 2
True Positives: 5
True Negatives: 136
False Positives: 38
False Negatives: 36


In [90]:
train_generator.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [91]:
model.predict(train_generator)

14/14 [==============================] - 3s 238ms/step


array([[1.56271607e-17, 6.04273144e-20, 5.41031147e-34, 1.00000000e+00],
       [2.36243610e-19, 5.16215329e-22, 7.75090298e-38, 1.00000000e+00],
       [4.95927244e-19, 3.05386522e-10, 1.00000000e+00, 0.00000000e+00],
       [9.97166216e-01, 2.41873530e-03, 1.98541875e-05, 3.95233248e-04],
       [1.32637359e-08, 1.41371966e-05, 9.99985814e-01, 2.02720077e-20],
       [1.98197332e-16, 1.46972279e-09, 1.00000000e+00, 0.00000000e+00],
       [6.28525704e-12, 2.26466028e-07, 9.99999762e-01, 2.38618056e-29],
       [3.89421915e-15, 1.46031690e-17, 1.35789322e-31, 1.00000000e+00],
       [3.51955422e-12, 3.23115614e-07, 9.99999642e-01, 8.37650033e-31],
       [8.50660916e-28, 8.76333860e-32, 0.00000000e+00, 1.00000000e+00],
       [9.99953747e-01, 4.29893444e-05, 2.51746215e-08, 3.18974799e-06],
       [9.99999881e-01, 1.07552751e-07, 1.23881439e-12, 2.46060883e-09],
       [9.98621345e-01, 1.16832415e-03, 8.41075325e-06, 2.01981136e-04],
       [7.63023330e-04, 9.99201834e-01, 3.50522714e

In [92]:
confusion_matrix

<tf.Tensor: shape=(4, 4), dtype=int32, numpy=
array([[12, 12,  8, 16],
       [12,  7, 12, 15],
       [ 8,  9,  5, 19],
       [14, 19, 18, 29]], dtype=int32)>

In [93]:
np.sum(np.diag(confusion_matrix))/np.sum(confusion_matrix)

0.24651162790697675

In [94]:
predictions = model.predict_generator(validation_generator)

predicted_labels = np.argmax(predictions, axis=1)

true_labels=validation_generator.classes

confusion_matrix = tf.math.confusion_matrix(true_labels, predicted_labels)

# Extract the true positives, true negatives, and false positives
true_positives = np.diag(confusion_matrix)
true_negatives = np.sum(confusion_matrix) - np.sum(confusion_matrix, axis=0) - np.sum(confusion_matrix, axis=1) + true_positives
false_positives = np.sum(confusion_matrix, axis=0) - true_positives
false_negatives = np.sum(confusion_matrix, axis=1) - true_positives

# Print the results for each class
for i in range(3):
    print("Class", i)
    print("True Positives:", true_positives[i])
    print("True Negatives:", true_negatives[i])
    print("False Positives:", false_positives[i])
    print("False Negatives:", false_negatives[i])

<ipython-input-94-af8a9803e842>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(validation_generator)


Class 0
True Positives: 6
True Negatives: 34
False Positives: 12
False Negatives: 9
Class 1
True Positives: 1
True Negatives: 47
False Positives: 6
False Negatives: 7
Class 2
True Positives: 5
True Negatives: 41
False Positives: 7
False Negatives: 8


In [95]:
confusion_matrix

<tf.Tensor: shape=(4, 4), dtype=int32, numpy=
array([[ 6,  1,  1,  7],
       [ 2,  1,  1,  4],
       [ 4,  1,  5,  3],
       [ 6,  4,  5, 10]], dtype=int32)>

In [96]:
# Count false positives for each class
false_positives = np.zeros(3)  # Initialize an array to store false positives for each class

for i in range(3):
    false_positives[i] = np.sum((predicted_labels == i) & (true_labels != i))

# Print false positives for each class
for i in range(3):
    print("False positives for class", i, ":", false_positives[i])

False positives for class 0 : 12.0
False positives for class 1 : 6.0
False positives for class 2 : 7.0


In [97]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = load_img(path, target_size=(224, 224))
  x = img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=16)
  print(fn)
  print(classes)

In [98]:
model.save('./Mbako/MyDrive/Project Bangkit/SaveModelV4/model.pb')
model.save('./Mbako/MyDrive/Project Bangkit/SaveModelV4/model.h5')


In [99]:
# EXERCISE: Use the tf.saved_model API to save your model in the SavedModel format. 
export_dir = './Mbako/MyDrive/Project Bangkit/SaveModelV4'

# YOUR CODE HERE
tf.saved_model.save(model,export_dir=export_dir)


In [100]:
model.save('./Mbako/MyDrive/Project Bangkit/SaveModelV4/model')

In [101]:
my_model = tf.keras.models.load_model('./Mbako/MyDrive/Project Bangkit/SaveModelV4/model')

In [102]:
# EXERCISE: Use the TFLiteConverter SavedModel API to initialize the converter
# YOUR CODE HERE
converter = tf.lite.TFLiteConverter.from_keras_model(my_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

In [103]:
tflite_model_file = pathlib.Path('./model.tflite')
tflite_model_file.write_bytes(tflite_model)

15162600

In [104]:
open('./Mbako/MyDrive/Project Bangkit/SaveModelV4/Model.tflite','wb').write(tflite_model)


15162600